In [99]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/_j/4p0l9p3s60q8v49n24yx3xn00000gn/T/ipykernel_19249/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [100]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [101]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/alex/Library/Mobile Documents/iCloud~md~obsidian/Documents/AX /Projects/Ironhack/Week4/Day3/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
data = data.head(1000)

# Modify for final system
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [102]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [103]:
# Your code

from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

print("Training set shape:", data_train.shape)
print("Test set shape:", data_test.shape)


Training set shape: (800, 2)
Test set shape: (200, 2)


## Data Preprocessing

In [104]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [105]:
import re
import pandas as pd

def clean_html_js(text: str) -> str:
    """
    Cleans HTML tags, JavaScript, and extra characters from a string.
    Returns clean plain text.
    """

    # Ensure the input is a string to avoid errors
    text = str(text)

    # 1. Remove HTML comments <!-- ... -->
    text = re.sub(r"<!--[\s\S]*?-->", "", text, flags=re.I)

    # 2. Remove <script>...</script> blocks
    text = re.sub(r"<script\b[^>]*>[\s\S]*?</script>", "", text, flags=re.I)

    # 3. Remove <style>...</style> blocks
    text = re.sub(r"<style\b[^>]*>[\s\S]*?</style>", "", text, flags=re.I)

    # 4. Remove inline event handlers like onclick="..."
    text = re.sub(r"\son\w+\s*=\s*(?:\"[^\"]*\"|'[^']*'|[^\s>]+)", "", text, flags=re.I)

    # 5. Remove javascript: URLs inside href or src attributes
    text = re.sub(r"\s(?:href|src)\s*=\s*([\"'])\s*javascript:[\s\S]*?\1", "", text, flags=re.I)

    # 6. Remove any remaining HTML tags <...>
    text = re.sub(r"</?[^>]+>", "", text)

    # 7. Remove stray < and > symbols
    text = re.sub(r"[<>]", "", text)

    # 8. Collapse multiple spaces into a single space and trim leading/trailing spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

# Apply the cleaning function to the 'text' column
data["text"] = data["text"].apply(clean_html_js)

In [106]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [107]:
import re
import pandas as pd

def clean_text_extra(text: str) -> str:
    """
    Further cleans text by removing special characters, numbers,
    single characters, and formatting for NLP readiness.
    """

    # Ensure we are working with a string
    text = str(text)

    # 1. Remove all special characters (keep only letters, numbers, and spaces)
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)

    # 2. Remove numbers
    text = re.sub(r"\d+", "", text)

    # 3. Remove all single characters (like "a", "b", "c" standing alone)
    text = re.sub(r"\b[A-Za-z]\b", "", text)

    # 4. Remove single characters from the start of the text
    text = re.sub(r"^[A-Za-z]\s+", "", text)

    # 5. Replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)

    # 6. Remove prefixed 'b' (like in bHello)
    text = re.sub(r"\bb([A-Za-z]+)", r"\1", text)

    # 7. Convert everything to lowercase
    text = text.lower()

    return text

data["text"] = data["text"].apply(clean_text_extra)

data.head() 

,text,label
0,dear sir strictly private business proposal am...,1
1,will do,0
2,noracheryl has emailed dozens of memos about h...,0
3,dear sirfmadamc know that this proposal might ...,1
4,fyi,0


## Now let's work on removing stopwords
Remove the stopwords.

In [108]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Remove stopwords from the 'text' column
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

data.head()

,text,label
0,dear sir strictly private business proposal mi...,1
1,,0
2,noracheryl emailed dozens memos haiti weekend ...,0
3,dear sirfmadamc know proposal might e surprise...,1
4,fyi,0


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [109]:
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Apply lemmatization to each word in the 'text' column
data['text'] = data['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

data.head()

[nltk_data] Downloading package wordnet to /Users/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,label
0,dear sir strictly private business proposal mi...,1
1,,0
2,noracheryl emailed dozen memo haiti weekend pl...,0
3,dear sirfmadamc know proposal might e surprise...,1
4,fyi,0


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [110]:
print(data['label'].unique())

[1 0]


In [111]:
# Top 10 words in ham messages
ham_words = ' '.join(data[data['label'] == 1]['text']).split()
ham_freq = pd.Series(ham_words).value_counts().head(10)
print(ham_freq)

# Top 10 words in spam messages
spam_words = ' '.join(data[data['label'] == 0]['text']).split()
spam_freq = pd.Series(spam_words).value_counts().head(10)
print(spam_freq)

e              1125
money           920
account         794
fund            703
u               550
transaction     416
country         406
transfer        392
ank             390
million         385
Name: count, dtype: int64
e            171
u            117
pm           115
would        106
state        103
ut            97
president     94
call          91
time          84
percent       77
Name: count, dtype: int64


## Extra features

In [117]:
# Use the 'text' column directly for feature engineering

money_simbol_list = "|".join(["euro", "dollar", "pound", "€", r"\$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

data_train['money_mark'] = data_train['text'].str.contains(money_simbol_list, case=False, regex=True).astype(int)
data_train['suspicious_words'] = data_train['text'].str.contains(suspicious_words, case=False, regex=True).astype(int)
data_train['text_len'] = data_train['text'].apply(len)

data_val['money_mark'] = data_val['text'].str.contains(money_simbol_list, case=False, regex=True).astype(int)
data_val['suspicious_words'] = data_val['text'].str.contains(suspicious_words, case=False, regex=True).astype(int)
data_val['text_len'] = data_val['text'].apply(len)

data_train.head()

,text,label,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,0,0,106
535,I have not been able to reach oscar this am. W...,0,0,0,101
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,0,0,141
557,I can have it announced here on Monday - can't...,0,0,0,52
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,1,1,1750


## How would work the Bag of Words with Count Vectorizer concept?

In [113]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['text'])

# Transform the test data
X_val_tfidf = tfidf_vectorizer.transform(data_val['text'])

# Print the shape of the vectorized dataset
print("Train shape:", X_train_tfidf.shape)
print("Test shape:", X_val_tfidf.shape)

Train shape: (800, 23374)
Test shape: (200, 23374)


## And the Train a Classifier?

In [115]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [116]:
# Your code